In [2]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128

Looking in indexes: https://download.pytorch.org/whl/cu128
  Using cached https://download.pytorch.org/whl/filelock-3.13.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached https://download.pytorch.org/whl/sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached https://download.pytorch.org/whl/networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached https://download.pytorch.org/whl/fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
  Using cached https://download.pytorch.org/whl/pillow-11.0.0-cp313-cp313-win_amd64.whl.metadata (9.3 kB)
  Using cached https://download.pytorch.org/whl/mpmath-1.3.0-py3-none-any.whl (536 kB)
   ---------------------------------------- 0.0/3.3 GB ? eta -:--:--
   ---------------------------------------- 0.0/3.3 GB ? eta -:--:--
   ---------------------------------------- 0.0/3.3 GB 2.3 MB/s eta 0:23:16
   ---------------------------------------- 0.0/3.3 GB 2.4 MB/s eta 0:22:24
   ---------------------------------------- 0.0/3.3 GB 2.7 MB/s eta 0:

In [3]:
import torch
import torchvision

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class PhysionetTransformer(nn.Module):
    """
    Transformer-based encoder for the Physionet dataset.
    This model is adapted to handle longer sequences and a different feature dimension.
    """
    def __init__(self, input_dim=6, d_model=128, nhead=4, num_layers=3, num_classes=3):
        """
        Initializes the PhysionetTransformer model.

        Args:
            input_dim (int): The number of features in the input data (default: 6).
            d_model (int): The dimensionality of the model's embeddings (default: 128).
            nhead (int): The number of heads in the multi-head attention mechanism (default: 4).
            num_layers (int): The number of Transformer encoder layers (default: 3).
            num_classes (int): The number of output classes for classification (default: 3).
        """
        super().__init__()
        # Project raw features to model dimension
        self.embed = nn.Linear(input_dim, d_model)

        # Stacked Transformer encoder layers
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=256,
            dropout=0.1,
            batch_first=False # Expects (Seq_Len, Batch, Features)
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # Classification head on pooled CLS-like token
        self.classifier = nn.Linear(d_model, num_classes)

    def forward(self, x):
        """
        Forward pass of the model.

        Args:
            x (torch.Tensor): Input tensor of shape (Batch, Seq_Len, Features), e.g., [B, 1000, 6].

        Returns:
            torch.Tensor: The output logits of shape (Batch, num_classes).
            torch.Tensor: The extracted class token embedding of shape (Batch, d_model).
        """
        # x: [B, T=1000, 6]
        h = self.embed(x)               # -> [B, 1000, 128]
        h = h.permute(1, 0, 2)          # -> [T=1000, B, 128] for Transformer
        out = self.transformer(h)       # -> [T=1000, B, 128]
        cls_token = out[0]              # Use the output of the first time step as the summary token
        logits = self.classifier(cls_token) # -> [B, 3]
        return logits, cls_token

In [5]:
phy = PhysionetTransformer()
print(phy)

PhysionetTransformer(
  (embed): Linear(in_features=6, out_features=128, bias=True)
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-2): 3 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (classifier): Linear(in_features=128, out_features=3, bias=True)
)


c:\Users\jyoti\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\nn\modules\transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
